In [ ]:
#Import libraries
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np
from datetime import date

# Adjust style
plt.style.use('seaborn-whitegrid')

In [ ]:
# Set up model variables
# API Key from EIA
api_key = 'dc1de54d5800a76b0e84950e9eef64e7'

# PADD Names to Label Columns
PADD_NAMES = ['PADD 1','PADD 2','PADD 3','PADD 4','PADD 5']
PADD_KEY = ['PET.MCRRIP12.M','PET.MCRRIP22.M','PET.MCRRIP32.M','PET.MCRRIP42.M','PET.MCRRIP52.M']

In [ ]:
# Initialize list
final_data = []

# Choose start and end dates
startDate = '2009-01-01'
endDate = '2021-01-01'

In [ ]:
# Pull in data via EIA API
for i in range(len(PADD_KEY)):
          #http://api.eia.gov/series/?api_key=YOUR_API_KEY_HERE&series_id=PET.MCRRIP32.M
    url = 'http://api.eia.gov/series/?api_key=' + api_key + '&series_id=' + PADD_KEY[i]
    
    r = requests.get(url)
    json_data = r.json()
    
    if r.status_code == 200:
        print('Success!')
    else:
        print('Error')
    
    df = pd.DataFrame(json_data.get('series')[0].get('data'), columns = ['Date', PADD_NAMES[i]])
    df.set_index('Date', drop=True, inplace=True)
    final_data.append(df)

In [ ]:
# Create final dataframeby concatenating all the df in final_data together
crude_input_monthly = pd.concat(final_data, axis=1)

In [ ]:
# Create date as datetype datatype
crude_input_monthly['Year'] = crude_input_monthly.index.astype(str).str[:4]
crude_input_monthly['Month'] = crude_input_monthly.index.astype(str).str[4:]
crude_input_monthly['Day'] = 1
crude_input_monthly['Date'] = pd.to_datetime(crude_input_monthly[['Year','Month','Day']])
crude_input_monthly.set_index('Date',drop=True,inplace=True)
crude_input_monthly.sort_index(inplace=True)
crude = crude_input_monthly[startDate:endDate]

In [ ]:
# Read in IIR Weekly Data
df = pd.read_csv('L:\FFX\intl\Trading\Balances\Turnarounds\TAoutputM.csv')

In [ ]:
# Only take USA
df = df[df['COUNTRY'] == 'U.S.A.']

# Remove cancelled data
df = df[df['OUTAGE_STA'] != 'Cancelled']

# Select only crude units
df = df[df['UTYPE_DESC'] == 'Atmospheric Distillation']

# Make day column = 1 so we can groupby month of each year
df['Day'] = 1

# Create MonthYear column
df['MonthYear'] = pd.to_datetime(df[['Year','Month','Day']])

df_view = df[df['Category'] != 'IIR Overridden']

# Filter by dates
mask = (df_view['MonthYear'] >= startDate) & (df_view['MonthYear'] <= endDate)
df_view = df_view.loc[mask]
df_view.set_index('MonthYear', inplace=True)

df_pivot = pd.pivot_table(df_view,
                          values=['CrudeLoss'],
                          index=['MonthYear'],
                          columns=['PAD_DIST'],
                          aggfunc=np.sum)/1000

df_pivot = df_pivot.fillna(0)
df_pivot = df_pivot.round(decimals=0)
df_pivot.columns = df_pivot.columns.droplevel(0)
df_pivot.columns = ['P1 Crude Loss','P2 Crude Loss','P3 Crude Loss','P4 Crude Loss','P5 Crude Loss']

In [ ]:
# Set columns to create 5 or 10 year average by month
final = crude.join(df_pivot, how='outer')
final['Year'] - final.index.year
final['Month'] - final.index.month
final['Day'] - final.index.day
final['P1 Avg'] = np.nan
final['P2 Avg'] = np.nan
final['P3 Avg'] = np.nan
final['P4 Avg'] = np.nan
final['P5 Avg'] = np.nan

In [ ]:
# Choose 5 or 10 year average

choice = '5 Year Average' # '10 Year Average'
current_year = date.today().year

if choice == '5 Year Average':
    year_list = [current_year-6, current_year-5, current_year-4, current_year-3, current_year-2, current_year-1, 
                 current_year, current_year+1]
elif choice == "10 Year Average":
    year_list = [current_year-11, current_year-10, current_year-9, current_year-8, 
                 current_year-7, current_year-6, current_year-5, current_year-4, 
                 current_year-3, current_year-2, current_year-1, current_year, current_year+1]
else:
    print("Error: Choose 5 or 10 year range")
    
for i in range(len(yearlist)):
    a = final[(final['Year'] >= yearlist[i]-5) & (final['Year'] < yearlist[i])].groupby('Month').mean()
    a = a.iloc[:,0:5]
    a.columns = ['P1 Avg', 'P2 Avg', 'P3 Avg', 'P4 Avg', 'P5 Avg']
    a.set_index(final[str(yearlist[i])].index, drop=True)
    
    final.update(a, overwrite=True)

In [ ]:
# Create Pct Change
final['P1 PctChg'] = final['P1 Avg'].pct_change()
final['P2 PctChg'] = final['P2 Avg'].pct_change()
final['P3 PctChg'] = final['P3 Avg'].pct_change()
final['P4 PctChg'] = final['P4 Avg'].pct_change()
final['P5 PctChg'] = final['P5 Avg'].pct_change()

In [ ]:
# Begin Regression
# Create variable models for each PADD
P1_Var = # Proprietary variables here
P2_Var = # Proprietary variables here
P3_Var = # Proprietary variables here
P4_Var = # Proprietary variables here
P5_Var = # Proprietary variables here

padd_var = [P1_Var, P2_Var, P3_Var, P4_Var, P5_Var]
padd_pred = ['P1 Prediction', 'P2 Prediction', 'P3 Prediction', 'P4 Prediction', 'P5 Prediction']

# Create data to make the model
test_data = final.dropna()

# Create model and predictions
for j in range(len(padd_var)):
    X = test_data[padd_var[j]]
    y = test_data[PADD_NAMES[j]]
    X = sm.add_constant(X)
    
    # Note the differences in argument order
    model = sm.OLS(y, X).fit()
    final[padd_pred[j]] = model.predict(X)
    
    z = final[final[PADD_NAMES[j]].isnull() == True]
    
    z[padd_predd[j]] = # Proprietary formula
    
    final.update(z, overwrite = True)

In [ ]:
# Plot the predictions
plotdata = final[str(current_year-5):str(current_year+2)]

for k in range(len(PADD_NAMES)):
    ax1 = plotdata[PADD_NAMES[k]].plot()
    ax2 = plotdata[padd_pred[k]].plot()
    plt.title(PADD_NAMES[k] + 'Crude Runs')
    plt.legend()
    plt.savefig(PADD_NAMES[k]+ ' Crude Runs.jpg')
    plt.show()